### Documentation

To read more about the search API, visit the [docs](https://www.elastic.co/docs/reference/elasticsearch/mapping-reference/dense-vector)

### Connect to ElasticSearch

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'FQt-ffZfTpeh0Snf3pUAQw',
 'name': 'ae8b5b4be42b',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


### Inserting documents

Because we are using the `dense_vector` data type, we need to do the mapping manually.

In [2]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(
    index="my_index",
    mappings={
        "properties":{
            "sides_length": {
                "type":"dense_vector",
                "dims":4
            },
            "shape" :{
                "type":"keyword"
            }
        }
    },
)
                

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})